In [13]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, BitsAndBytesConfig
from tqdm.notebook import tqdm

# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
!pip install transformers
!pip install torch
!pip install tpdm

In [ ]:
!pip install torch torchvision torchaudio
!pip install datasets

In [ ]:
!pip install wandb

# 1. 데이터 수집 및 준비

In [18]:
train_data = pd.read_csv('/kaggle/input/train-data2/0523_942292row.csv').iloc[:,1:]
train_data = train_data.loc[train_data.standard_form.notnull()]

/tmp/ipykernel_34/2456427794.py:1: DtypeWarning: Columns (0,2,4,5,6,7,8,9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/kaggle/input/train-data2/0523_942292row.csv').iloc[:,1:]


(942292, 17)


In [19]:
eval_data = train_data.sample(n=500)
train_data = train_data.drop(eval_data.index, errors='ignore')
    

(941792, 17)


# 2. 모델 선택 및 수정
- 모델은 KoBART 모델을 사용하고 싶음
- 한국어로 학습되어 있고 무게가 그렇게 무겁지 않아 사용하기에 적절하다고 판단
- URL : gogamza/kobart-base-v2

In [20]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# 모델과 토크나이저 로드
checkpoint_path = '/kaggle/input/model-train-ver2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(checkpoint_path)
model = BartForConditionalGeneration.from_pretrained(checkpoint_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [21]:
## 제주어 표준어 구분
## 이거 사실 안해도 되는데 True False 보다는 string 이 나을거 같아서 그냥 변환
train_data.isDialect = train_data.isDialect.apply(lambda x : '제주' if x == True else '표준')


## 제주 발화만 남기기
## 사실 isDialect가 표준인 경우에는 두 문장이 모두 표준어이기 때문에 source, target 문장이 같아서 의미가 없다고 생각해 제거
train_data = train_data.loc[train_data.isDialect == '제주']
#train_data = train_data.sample(n=600000)
print(train_data)

          year topic  speaker_id  age sex    occupation birthplace education  \
0       2020.0    가족         1.0  50대  여성  전문가 및 관련 종사자         제주        대졸   
1       2020.0    가족         1.0  50대  여성  전문가 및 관련 종사자         제주        대졸   
2       2020.0    가족         1.0  50대  여성  전문가 및 관련 종사자         제주        대졸   
3       2020.0    가족         1.0  50대  여성  전문가 및 관련 종사자         제주        대졸   
4       2020.0    가족         1.0  50대  여성  전문가 및 관련 종사자         제주        대졸   
...        ...   ...         ...  ...  ..           ...        ...       ...   
832461  2021.0    게임         1.0  30대  여성  전문가 및 관련 종사자         제주    대학원 이상   
832462  2021.0    게임         3.0  30대  여성        사무 종사자         제주        대졸   
832463  2021.0    게임         3.0  30대  여성        사무 종사자         제주        대졸   
832464  2021.0    게임         2.0  30대  여성      무직/취업준비생         제주    대학원 이상   
832465  2021.0    게임         3.0  30대  여성        사무 종사자         제주        대졸   

                                       

In [22]:
## 제주어 토큰, 표준어 토큰 정의하기
jeju_token = "[제주]"
standard_token = "[표준]"

## 양방향 데이터 리스트 생성
bidirectional_data = []

for dialect, standard in zip(train_data['dialect_form'], train_data['standard_form']):
    ## 토큰이 [제주] 일 경우 제주어 -> 표준어
    bidirectional_data.append({
        "source": jeju_token + " " + dialect,
        "target": standard
    })
    ## 토큰이 [표준] 일 경우 표준어 -> 제주어
    bidirectional_data.append({
        "source": standard_token + " " + standard,
        "target": dialect
    })

## 데이터 토크나이징
tokenized_data = []
for item in bidirectional_data:
    source_encodings = tokenizer(item['source'], max_length=64, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(item['target'], max_length=64, truncation=True, padding="max_length", return_tensors="pt")
    tokenized_data.append({
        "input_ids": source_encodings["input_ids"],
        "attention_mask": source_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    })
## 데이터가 어떻게 토큰화되었는지 한번 확인해보슈


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

formatted_data_df = pd.DataFrame([{
    "input_ids": np.array(fd["input_ids"].numpy().tolist()[0], dtype=np.uint16),
    "attention_mask": np.array(fd["attention_mask"].numpy().tolist()[0], dtype=np.uint8),
    "labels": np.array(fd["labels"].numpy().tolist()[0], dtype=np.uint16)
} for fd in tokenized_data])

## 변환된 데이터를 DataFrame으로 변환
#formatted_data_df = pd.DataFrame([{
#    "input_ids": fd["input_ids"].numpy().tolist()[0],  ## Tensor를 리스트로 변환
#    "attention_mask": fd["attention_mask"].numpy().tolist()[0],  ## Tensor를 리스트로 변환
#    "labels": fd["labels"].numpy().tolist()[0]  ## Tensor를 리스트로 변환
#} for fd in tokenized_data])

## 데이터를 Dataset 형식으로 변환
train_dataset = Dataset.from_pandas(formatted_data_df)

In [9]:
## 학습 데이터셋을 학습 및 평가용으로 분리 (예: 95% 학습, 5% 평가)
train_test_split = train_dataset.train_test_split(test_size=0.05)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# 3. 모델 학습 및 fine-tuning

In [ ]:
#! pip install -U accelerate
#! pip install -U transformers

import accelerate
import transformers

transformers.__version__, accelerate.__version__

## 3-1. 기존 모델에 추가 학습하는 경우

In [ ]:
#check120000 이후 
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='/kaggle/working/chatbot',          # output directory for model checkpoints    
    learning_rate=2e-5,   
    num_train_epochs=4,              # total number of training epochs
    predict_with_generate=True,   
    per_device_train_batch_size=16,   # batch size per device during training
    warmup_steps=15000,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000,
    save_total_limit=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",           # Save checkpoint every specified number of steps
    resume_from_checkpoint=checkpoint_path, #이전에 저장된 체크포인트부터 학습 재시작
    load_best_model_at_end=True,     # 학습이 종료될 때 최적 모델을 로드
    metric_for_best_model="eval_loss",  # 최적 모델을 결정하는 메트릭
)

import wandb
from transformers import AdamW,get_linear_schedule_with_warmup,EarlyStoppingCallback

optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps, 
    num_training_steps=197608  # 총 훈련 스텝 수
)
# API 키를 직접 입력
wandb.login(key="")

trainer = Seq2SeqTrainer(
    model=model,                         ## 학습할 모델
    args=training_args,                  ## 학습 설정
    train_dataset=dataset_dict['train'], ## 학습 데이터셋
    eval_dataset=dataset_dict['test'],   ## 평가 데이터셋
    tokenizer=tokenizer,
    optimizers=(optimizer,scheduler), #AdamW
    callbacks=[EarlyStoppingCallback(  # 조기 종료 콜백
        early_stopping_patience=1,
        early_stopping_threshold=None,  # 옵션: 최소 향상 정도
    )]
)

## 학습 시작
trainer.train()

## 학습 종료
wandb.finish()

## 3-2. 모델에 새로 학습하는 경우

In [ ]:
#새로운 모델 학습 버전
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
#처음부터 다시 모델 학습 용도
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='/kaggle/working/chatbot',          # output directory for model checkpoints    
    learning_rate=2e-5,   
    predict_with_generate=True,   
    per_device_train_batch_size=16,   # batch size per device during training
    warmup_steps=15000,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000,
    num_train_epochs=4,              # total number of training epochs
    save_total_limit=4,
    save_strategy="epoch"           # Save checkpoint every specified number of steps
)

import wandb
from transformers import AdamW,get_linear_schedule_with_warmup

# API 키를 직접 입력
wandb.login(key="aff67fd137854b3632153f22c1ff7d40506aa349")
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=training_args.warmup_steps, 
    num_training_steps=395424  # 총 훈련 스텝 수
)

trainer = Seq2SeqTrainer(
    model=model,                         ## 학습할 모델
    args=training_args,                  ## 학습 설정
    train_dataset=dataset_dict['train'], ## 학습 데이터셋
    eval_dataset=dataset_dict['test'],   ## 평가 데이터셋
    tokenizer=tokenizer,
    optimizers=(optimizer,scheduler) #AdamW
)

## 학습 시작
trainer.train()

## 학습 종료
wandb.finish()

# 4. 모델 평가

In [ ]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, BitsAndBytesConfig
from tqdm.notebook import tqdm

# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast,AutoModelForSeq2SeqLM
from transformers import AutoModelForMaskedLM
# 모델과 토크나이저의 경로
model_path = '/kaggle/input/model-0524-12000' # 본인 모델 경로 설정

# 모델과 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)

model.to(device)

In [ ]:
## 데이터 프레임 로드 및 준비
#df_val = pd.read_csv("/kaggle/input/bleu-ver1/.csv")

df_val = eval_data #맨 처음 단계에서 500개 미리 추출 후 train_data에서 삭제함 

#df_val = df_val.head(500)
#df_val=df_val.sample(n=500)
## 번역 Task의 문장만 남기기 & 결측치 제거
df_val = df_val.loc[df_val.isDialect == True]
df_val = df_val.dropna(subset=['standard_form', 'dialect_form'])

## 불용어 제거
df_val.form_샵 = df_val.form.apply(lambda x : 1 if '#' in x else 0)
df_val = df_val.loc[df_val.form_샵 == 0]
df_val.form_앤드 = df_val.form.apply(lambda x : 1 if '&' in x else 0)
df_val = df_val.loc[df_val.form_앤드 == 0]

## 문장을 4 ~ 64로 조정
df_val['form_len'] = df_val.standard_form.apply(lambda x : len(str(x)))
df_val = df_val.loc[(3 < df_val['form_len']) & (df_val['form_len'] <= 64)]

df_val

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from transformers import PreTrainedTokenizerFast
from transformers import pipeline

# 번역을 위한 파이프라인 생성
translation_pipeline = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=0,
    max_length = 64
)

jeju_token = "[제주]"
standard_token = "[표준]"

# 모든 번역을 저장할 리스트
dialect_targets = []
standard_targets = []
dialect_predictions = []
standard_predictions = []

for index, row in df_val.iterrows():
    dialect_form = row['dialect_form']
    standard_form = row['standard_form']

    ## 제주어 -> 표준어 번역 수행
    predicted_sentence_dialect = translation_pipeline(jeju_token + " " + dialect_form)[0]['translation_text']

    ## 표준어 -> 제주어 번역 수행
    predicted_sentence_standard = translation_pipeline(standard_token + " " + standard_form)[0]['translation_text']

    # 토크나이저를 사용하여 문장을 토큰화
    dialect_form_tokens = tokenizer.tokenize(dialect_form)
    standard_form_tokens = tokenizer.tokenize(standard_form)
    predicted_tokens_dialect = tokenizer.tokenize(predicted_sentence_dialect)
    predicted_tokens_standard = tokenizer.tokenize(predicted_sentence_standard)

    # 토큰화된 문장을 리스트에 추가
    dialect_targets.append([dialect_form_tokens])  # 참조는 리스트의 리스트가 되어야 함
    standard_targets.append([standard_form_tokens])  # 참조는 리스트의 리스트가 되어야 함
    dialect_predictions.append(predicted_tokens_dialect)
    standard_predictions.append(predicted_tokens_standard)

# corpus_bleu 함수를 사용한 BLEU 점수 계산
from_jeju_to_standard_bleu_score = corpus_bleu(standard_targets, dialect_predictions)
from_standard_to_jeju_bleu_score = corpus_bleu(dialect_targets, standard_predictions)
print(f"제주어 -> 표준어 번역 BLEU Score : {from_jeju_to_standard_bleu_score}")
print(f"표준어 -> 제주어 번역 BLEU Score : {from_standard_to_jeju_bleu_score}")

In [ ]:
input=df_val['standard_form'].sample(n=3)
input2=df_val['dialect_form'].sample(n=3)

In [ ]:
from transformers import pipeline

# 번역을 위한 파이프라인 생성
translation_pipeline = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=0
)

jeju_token = "[제주]"
standard_token = "[표준]"

# 특정 문장 번역 예시
for input_sentence in input:
    translated_sentence = translation_pipeline(standard_token + " " + input_sentence)[0]['translation_text']
    print('input_sentence :', input_sentence)
    print("표준어 -> 제주어:", translated_sentence)
    print()

for input_sentence in input2:
    translated_sentence = translation_pipeline(jeju_token + " " + input_sentence)[0]['translation_text']
    print('input_sentence :', input_sentence)
    print("제주어 -> 표준어:", translated_sentence)
    print()